## Installing necessary libraries

In [ ]:
%%bash

# Dependencies
pip install docx2txt
pip install strsimpy
pip install python-docx
pip install pandas

if ls docx2csv >/dev/null 2>&1; then
    echo "docx2csv exists."
else
    echo "Folder does not exist. Cloning docx2csv."
    git clone https://github.com/ivbeg/docx2csv.git
fi

In [ ]:
%%bash

source .env
cd docx2csv && echo "$PASSWORD" | sudo -S python3 setup.py install

In [ ]:
# ----- TEST DATA INPUT -----

# Computer Science Test Data.
# CURRENT_MAPPING="Lists_ComputerScience.docx"
# ORIGINAL_MAPPING="Original-Mapping-ComputerScience.csv"

# InformationSecurity Test Data.
CURRENT_MAPPING="Lists_MonashEngineering.docx"
ORIGINAL_MAPPING="Original-Mapping-MonashEngineering.csv"

In [ ]:
# importing necessary libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [ ]:
# extract tables from word document
from docx2csv import extract_tables, extract
tables = extract_tables(CURRENT_MAPPING)

In [ ]:
from docx import Document
document = Document(CURRENT_MAPPING)

In [ ]:
def read_docx_table(document,table_num):
  table = document.tables[table_num-1]
  data = [[cell.text for cell in row.cells] for row in table.rows]
  df = pd.DataFrame(data)
  return df

## PLO TABLE

In [ ]:
# Creating a dataframe for PLOs and it will accept 'n' number of PLOs
table_num=1
df = read_docx_table(document,table_num)
df.head(n=12)

In [ ]:
q1 = df.copy()
q1

In [ ]:
# assigning count vectorizer
count_vectorizer = CountVectorizer(stop_words='english', min_df=0.005)

In [ ]:
# Remove integers

# Data preprocessing for PLO dataframe
q1[1] = q1[1].str.lower()
corpus = q1[1].tolist()
corpii = count_vectorizer.fit_transform(corpus)
corpus

In [ ]:
corpii

In [ ]:
# extracting features names from PLO table
feature_names = count_vectorizer.get_feature_names_out()
feature_names

In [ ]:
len(feature_names)

In [ ]:
# Converting features to vector form and create a dataframe
X1 = pd.DataFrame(corpii.toarray(), columns=feature_names)

## CLO TABLE

In [ ]:
# Creating a dataframe for CLOs and it will accept 'n' number of CLOs
table_num=2
df1 = read_docx_table(document,table_num)
p1 = df1.copy()
p1

In [ ]:
# Data preprocessing for CLO dataframe
p1[1] = p1[1].str.lower()
corpus11 = p1[1].tolist()
corpii11 = count_vectorizer.fit_transform(corpus11)

In [ ]:
# extracting features names from CLO table
feature_names1 = count_vectorizer.get_feature_names_out()

In [ ]:
len(feature_names1)

In [ ]:
# Converting features to vector form and create a dataframe
X2 = pd.DataFrame(corpii11.toarray(), columns=feature_names1)
X2

In [ ]:
X2.head()

In [ ]:
# adding column index to the CLO table
U2 = pd.concat([df1[0], X2], axis=1)
U2.set_index(0, inplace=True)

In [ ]:
U2.head()

In [ ]:
# adding column index to the PLO table
U1 = pd.concat([df[0], X1], axis=1)
U1.set_index(0, inplace=True)

In [ ]:
U1

## Intersection method for both CLOs and PLOs

### Generalised list of words

In [ ]:
append_words = list(map(str.lower,['Cite', 'Define', 'Describe', 'Draw', 'Enumerate', 'Identify' 'Index', 'Indicate', 'Label', 'List', 'Match', 'Meet', 'Name', 'Outline', 'Point', 'Quote', 'Read', 'Recall', 'Recite', 'Recognize', 'Record', 'Repeat', 'Reproduce','Review',
'Select', 'State', 'Study', 'Tabulate', 'Trace', 'Write', 'Add', 'Approximate', 'Articulate', 'Associate', 'Characterize', 'Clarify', 'Classify', 'Compare', 'Compute', 'Contrast', 'Convert', 'Defend', 'Detail', 'Differentiate',
'Discuss', 'Distinguish', 'Elaborate', 'Estimate', 'Example', 'Explain', 'Express', 'Extend', 'Extrapolate', 'Factor', 'Generalize', 'Give', 'Infer', 'Interact', 'Interpolate', 'Interpret', 'Observe', 'Paraphrase', 'Picture graphically',
'Predict', 'Rewrite', 'Subtract', 'Summarize', 'Translate', 'Visualize', 'Acquire', 'Adapt', 'Allocate', 'Alphabetize', 'Apply', 'Ascertain', 'Assign', 'Attain', 'Avoid', 'Back up', 'Calculate', 'Capture', 'Change', 'Complete', 'Construct', 
'Customize', 'Demonstrate', 'Depreciate', 'Derive', 'Determine', 'Diminish', 'Discover', 'Employ', 'Examine', 'Exercise', 'Explore', 'Expose', 'Figure', 'Graph', 'Handle', 'Illustrate', 'Interconvert', 'Investigate', 'Manipulate', 'Modify', 
'Operate', 'Personalize', 'Plot','Practice', 'Prepare', 'Price', 'Process', 'Produce', 'Project', 'Provide', 'Relate', 'Round off', 'Sequence', 'Show', 'Simulate', 'Sketch', 'Solve', 'Subscribe', 'Transcribe', 'Use', 'Analyze', 'Audit', 
'Blueprint', 'Breadboard', 'Break down', 'Confirm', 'Correlate', 'Detect', 'Diagnose', 'Diagram', 'Discriminate', 'Dissect', 'Document', 'Ensure', 'Figure out', 'File', 'Group', 'Interrupt', 'Inventory', 'Layout', 'Manage', 'Maximize', 
'Minimize', 'Optimize', 'Order', 'Point out', 'Prioritize', 'Proofread', 'Query', 'Separate', 'Subdivide', 'Train', 'Transform', 'Appraise', 'Assess', 'Conclude', 'Counsel', 'Criticize', 'Critique', 'Evaluate', 'Grade', 'Hire', 'Judge', 
'Justify', 'Measure', 'Prescribe', 'Rank', 'Rate', 'Recommend', 'Release', 'Support', 'Test', 'Validate', 'Verify', 'Abstract', 'Animate', 'Arrange', 'Assemble', 'Budget', 'Categorize', 'Code', 'Combine', 'Compile', 'Compose', 'Cope', 
'Correspond', 'Create', 'Cultivate', 'Debug', 'Depict', 'Design', 'Develop', 'Devise', 'Dictate', 'Enhance', 'Facilitate', 'Format', 'Formulate', 'Generate', 'Import', 'Improve', 'Incorporate', 'Integrate', 'Interface', 'Join', 'Lecture', 
'Model', 'Network', 'Organize', 'Overhaul', 'Plan', 'Portray', 'Program', 'Rearrange', 'Reconstruct', 'Reorganize', 'Revise', 'Specify']))

In [ ]:
# using + operator to concat the generalised list of words to the PLO list
train_column = list(feature_names) + append_words

In [ ]:
# CLO list of words
test_column = feature_names1
test_column

In [ ]:
# Intersection method for extracting common column names from the tables (both CLO AND PLO)
# comparing whether the CLO column name is present in the PLO column names or not
train_column = list(feature_names) + append_words # (PLO table ) (# using + operator to concat PLO words and list of generalized words)
test_column = list(feature_names1)   # (CLO table)

In [ ]:
# This is the column names from both the tables (using intersection)
common_column = list(set(train_column).intersection(set(test_column)))
common_column

In [ ]:
print(common_column)

In [ ]:
len(common_column)

In [ ]:
# Filter the common column values from the CLO table
U3 = U2.filter(list(common_column), axis=1)

In [ ]:
U3.head()

In [ ]:
# extracting first row from PLO table and make a dataframe
Cs = []
for x in range(len(df)):
    Cs.append(U1.loc[['PO'+str(x+1)]])

In [ ]:
# Concatenating these extracted each PLOs with 'n' number of CLOs
Dds = []
for x in range(len(df)):
    Dds.append(pd.concat([Cs[x],U3], sort=True))

In [ ]:
# Filling the nan values of the concatenated dataframes
Ds = []
for x in range(len(df)):
    Ds.append(Dds[x].fillna(0))

## Calculate Cosine similarity

In [ ]:
# Calculate cosine similarity for concatenated dataframes and create a new dataframe
for x in range(len(df)):
    Dds[x] = pd.DataFrame(cosine_similarity(Ds[x], dense_output=True))
Ds

In [ ]:
# Extract the '0'th column because it has the CLO-PLO  cosine similarity values. We are neglecting the remaining ones.
# Renaming the '0'th column name to 'Pn' ['P1, P2, P3, P4, ... 'Pn']
for x in range(len(df)):
    Dds[x].rename(columns = {0 :'PO'+str(x+1)}, inplace = True)

Dds

In [ ]:
# Concatenating each  '0'th column from different cosine similarity dataframes
Ddn = []
for x in range(len(df)):
    Ddn.append(Dds[x]['PO'+str(x+1)])

d = pd.concat(Ddn, axis=1)
d

In [ ]:
# '0'th column gives us 1 which means each PLO map with own PLO.
# So we are removing that column.
dd = d[1:]
dd

In [ ]:
# resetting index
dd.reset_index(inplace = True)
dd.drop(['index'], axis=1, inplace = True)
dd

In [ ]:
# print the matrix
print(dd)

In [ ]:
## This code loads the vector file into the word_vectors variable
## Download the vector file from https://fasttext.cc/docs/en/english-vectors.html (first file on the website), unzip the file and store in your local development folder
## Note: This piece of code may take upto an hour or two to run depending on your pc specs.
## My i5 8th gen with 8gig ram took 58mins to run.

# from gensim.models import KeyedVectors

# # Path to the downloaded .vec file
# path_to_vectors = 'wiki-news-300d-1M.vec'
# # path_to_vectors = 'wiki.en.vec'
# # Load the word vectors
# word_vectors = KeyedVectors.load_word2vec_format(path_to_vectors)

# # Find similar words
# similar_words = word_vectors.most_similar('cat')

# # Calculate word similarity
# similarity = word_vectors.similarity('cat', 'dog')

# # Perform vector arithmetic
# result = word_vectors['king'] - word_vectors['man'] + word_vectors['woman']


In [ ]:
import pandas as pd
import spacy

file_path = "Bloom and SOLO Verbs.xlsx"

xls = pd.ExcelFile(file_path)
sheet_names = xls.sheet_names
# print(sheet_names)

sources = {} # Copy verbs from excel into var to maniupulate easier
verbs = {   # Complete list of verbs sorted into their taxonomy. Includes # of occurrences and the potential sources and levels the verb could be mapped to
    "Cognitive": {},
    "Affective": {},
    "Psychomotor": {},
    "SOLO": {}
}

domain_templates = {    # Template structure for each taxonomy
    "Cognitive": {
        "Remembering": set(),
        "Understanding": set(),
        "Applying": set(),
        "Analysing": set(),
        "Evaluating": set(),
        "Creating": set()
    },
    "Affective": {
        "Receiving": set(),
        "Responding": set(),
        "Valuing": set(),
        "Organisation": set(),
        "Characterisation": set(),
    },
    "Psychomotor": {
        "Perception": set(),
        "Set": set(),
        "Guided Response": set(),
        "Mechanism": set(),
        "Complex Overt Response": set(),
        "Adaptation": set(),
        "Origination": set()    
    },
    "SOLO": {
        "Prestructural": set(),
        "Unistructural": set(),
        "Multistructural": set(),
        "Relational": set(),
        "Extended Abstract": set()
    }
}

# Var to store final mappings of verbs for each taxonomy
mapped_verbs = domain_templates

# Read all sources and store count of verbs into var
for domain_key, domain_item in mapped_verbs.items():    # For each taxonomy
    for current_sheet_name in sheet_names:      # For each source sheet
        if domain_key in current_sheet_name:    # If the current taxonomy matches the current source sheet
            current_sheet = pd.read_excel(xls, current_sheet_name)  # Read the sheet into a var
            sources[current_sheet_name] = domain_templates.get(domain_key)  # Add the appropriate taxonomy template structure
            for j in range(len(domain_item.keys())):        # For each level of the taxonomy
                current_column = current_sheet.columns[j]
                column_values = current_sheet[current_column].values.tolist()
                for verb in range(len(column_values)):      # For each verb in the level
                    if type(column_values[verb])!= str or len(str(column_values[verb]).strip()) == 0:   # Validate the verb is a string
                        continue
                    v = str(column_values[verb]).strip().lower()    # Format verb
                    sources[current_sheet_name][current_column].add(v)
                    if v not in verbs:      # If this is the first instance of the verb for the taxonomy init the dict
                        verbs[domain_key][v] = {
                            "count": 1,
                            "potentials": [{
                                "level": current_column, 
                                "source": current_sheet_name
                                }]
                        }
                    else:   # Else add vars
                        verbs[domain_key][v]["count"] += 1
                        verbs[domain_key][v]["potentials"].append({
                                "level": current_column, 
                                "source": current_sheet_name
                                })

# print(sources)
# print(verbs)
# test = verbs
# solo = verbs["SOLO"]
# # print(solo)
# for verb in solo.items():
#     print(verb)


# Classify each verb in a taxonomy into a single level
for domain_key, domain_item in verbs.items():   # For each taxonomy
    for v in domain_item:   # For each verb
        if verbs[domain_key][v]["count"] == 1:  # If there is only one instance of the verb, map it
            mapped_verbs[domain_key][verbs[domain_key][v]["potentials"][0]["level"]].add(v)
        else:
            chosen_source = None
            chosen_level = None
            potential_sources = []
            potential_levels = []
            for l in verbs[domain_key][v]["potentials"]:    # Iterate through the potential level mappings
                if l["source"] not in potential_sources or l["level"] not in potential_levels:
                    potential_sources.append(l["source"])
                    potential_levels.append(l["level"])
            for sheet in sheet_names:   # For each source, map verb to first matching source (Assumes sources are sorted in order of priority for mapping)
                if sheet in potential_sources:
                    chosen_source = sheet
                    chosen_level = potential_levels[potential_sources.index(sheet)]
                    break
            mapped_verbs[domain_key][chosen_level].add(v)

## Function to identify verbs in a sentence
def identify_verbs(sentence):
    # Load the English language model in spaCy
    nlp = spacy.load('en_core_web_sm')
    
    # Process the sentence using spaCy
    doc = nlp(sentence)
    
    # Extract the verbs from the processed sentence
    verbs = [token.lemma_ for token in doc if token.pos_ == 'VERB']
    
    return verbs

In [ ]:
def bloom_mapping(sentences):

    mappings = []

    final_level = None
    passed_mappings = 0
    failed_mappings = 0
    total_mappings = len(sentences)
    for i in range(len(sentences)): # Iterates over the LOs
        identified_verbs = identify_verbs(sentences[i])

        score_list = {
            "Cognitive": {
                "Remembering": 0,
                "Understanding": 0,
                "Applying": 0,
                "Analysing": 0,
                "Evaluating": 0,
                "Creating": 0
            },
            "Affective": {
                "Receiving": 0,
                "Responding": 0,
                "Valuing": 0,
                "Organisation": 0,
                "Characterisation": 0
            },
            "Psychomotor": {
                "Perception": 0,
                "Set": 0,
                "Guided Response": 0,
                "Mechanism": 0,
                "Complex Overt Response": 0,
                "Adaptation": 0,
                "Origination": 0    
            },
            "SOLO": {
                "Prestructural": 0,
                "Unistructural": 0,
                "Multistructural": 0,
                "Relational": 0,
                "Extended Abstract": 0
            }
        }
        for taxonomy_key, taxonomy_item in mapped_verbs.items():
            for level in taxonomy_item.keys(): # Level
                for verb in mapped_verbs[taxonomy_key][level]: # Verb
                    similarity_score = 0
                    for l in range(len(identified_verbs)):
                        try:    # Currently some of the 'verbs' identified are phrases rather than words and it was throwing errors so this is a temp solution 
                            similarity_score += word_vectors.similarity(identified_verbs[l], verb)
                        except:
                            pass
                    score_list[taxonomy_key][level] += similarity_score

        # Identify level based on similarity
        max_score = {
            "Cognitive": { "Level": None, "Score": 0 },
            "Affective": { "Level": None, "Score": 0 },
            "Psychomotor": { "Level": None, "Score": 0 },
            "SOLO": { "Level": None, "Score": 0 }
        }
        for t_key, t_item in score_list.items():
            for l in t_item:
                if max_score[t_key]["Score"] < score_list[t_key][l]:
                    max_score[t_key] = { "Level": l, "Score": score_list[t_key][l] }
        mappings.append(max_score)

    return mappings

# TODO: Classify the verbs in each of the learning outcomes
ulo_sentences = p1[1].to_list()
plo_sentences = df[1].to_list()

ulo_classifications = bloom_mapping(ulo_sentences)
plo_classifications = bloom_mapping(plo_sentences)

# Iteration 1: Assume both CLOs and POs are classifiable into Cognitive, Affective, Psychomotor Levels. 
# If levels match, boost the CLO to PO coefficient by 0.1
# If levels different, don't boost

In [ ]:
# for classification in ulo_classifications:
#     print(classification)
for x in range(len(df)):
    for i in range(len(ulo_sentences)):
        if ulo_classifications[i]['Cognitive']['Level'] == plo_classifications[x]['Cognitive']['Level']:
            if dd['PO'+str(x+1)][i] + 0.15 <= 1:
                dd['PO'+str(x+1)][i] += 0.15 # Add Offset
            else:
                dd['PO'+str(x+1)][i] = 1
        else:
            if dd['PO'+str(x+1)][i] - 0.15 >= 0:
                dd['PO'+str(x+1)][i] -= 0.15 # Add Offset
            else:
                dd['PO'+str(x+1)][i] = 0

In [ ]:
# store the matrix into csv file
dd.to_csv('pseudocodematrix.csv', index=False)

## Setting threshold value (taking min and max of each column and divided by 2)
## threshold value = (min +max)/2

In [ ]:

# TODO: Alter the threshold based on matching hierarchy type & bloom verb instead of simply using (column_max+column_min)/2

# Setting threshold value 
# Taking min max average of each column and set that as a threshold value

# This will change the coefficients into 0 or 1 mappings in the dd dataframe
for x in range(len(df)):
    tes = dd['PO'+str(x+1)].values.min()
    tes1 = dd['PO'+str(x+1)].values.max()
    tt1 = (tes+tes1)/2
    
    if tt1 == 0:
      dd['PO'+str(x+1)] = dd['PO'+str(x+1)] 
    else:
      dd['PO'+str(x+1)] = dd['PO'+str(x+1)].apply(lambda x: 1 if x >= tt1 else 0)
# dd

In [ ]:
# dd

dd.to_csv('PLO-CLOmapping.csv', index=False)

In [ ]:
dd.head()

In [ ]:
# human generated output
d= pd.read_csv(ORIGINAL_MAPPING)
d.head()

In [ ]:
df3 = d.copy()

In [ ]:
dd

In [ ]:
df3

In [ ]:
for x in range(len(df)):
  df3['PO'+str(x+1)] = np.where(dd['PO'+str(x+1)] == df3['PO'+str(x+1)], 'True', 'False')

In [ ]:
df3.head()

In [ ]:
for x in range(len(df)):
  df3['PO'+str(x+1)] = df3['PO'+str(x+1)].replace('True', 1)
  df3['PO'+str(x+1)] = df3['PO'+str(x+1)].replace('False', 0)

In [ ]:
df3.head()

In [ ]:
# calculating accuracy of the table
df3['acc'] = df3.mean(axis=1)
df3.head(n=100)

In [ ]:
df4 = pd.concat([df1[0], df3], axis=1)
df4.head(n=100)

In [ ]:
df4.set_index(0, inplace=True)
df4.head(n=100)

df4.to_csv('WasMappingSuccessful.csv')

In [ ]:
df4['acc'].mean()